<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/OpenELM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/apple/OpenELM

https://huggingface.co/apple/OpenELM-3B

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q

In [44]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%cd /content/
!git lfs install
!git clone https://huggingface.co/apple/OpenELM-3B

In [ ]:
!echo $HUGGINGFACE_ACCESS_TOKEN_WRITE

In [46]:
!python /content/OpenELM-3B/generate_openelm.py --model apple/OpenELM-3B --hf_access_token $HUGGINGFACE_ACCESS_TOKEN_WRITE --prompt "Okay, how about a more detailed explanation to a high schooler?" --generate_kwargs repetition_penalty=1.2

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100% 3/3 [00:02<00:00,  1.48it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-12 18:10:44.338179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 18:10:44.338235: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for 

In [ ]:
#harness_repo="public-lm-eval-harness"
!git clone https://github.com/EleutherAI/lm-evaluation-harness/
%cd /content/lm-evaluation-harness
# use main branch on 03-15-2024, SHA is dc90fec
#!git checkout dc90fec
!pip install -e .
#!cd ..

In [ ]:
!pip install datasets@git+https://github.com/huggingface/datasets.git@66d6242 -q
!pip install tokenizers>=0.15.2 transformers>=4.38.2 sentencepiece>=0.2.0 -q

In [1]:
%mkdir /content/lm_eval_output

In [2]:
%cd /content/lm_eval_output

/content/lm_eval_output


In [36]:
import os
os.environ['hf_model'] = 'apple/OpenELM-3B'
os.environ['task']='mmlu,winogrande'
os.environ['shot']='5'

os.environ['tokenizer']='meta-llama/Llama-2-7b-hf'
os.environ['add_bos_token']='True'
os.environ['batch_size']='1'

In [37]:
!echo $hf_model
!echo $task
!echo $shot

!lm_eval --model hf --verbosity DEBUG --model_args pretrained=${hf_model},trust_remote_code=True,add_bos_token=${add_bos_token},tokenizer=${tokenizer} --tasks ${task} --device cuda:0 --num_fewshot ${shot} --output_path /content/lm_eval_output/${hf_model//\//_}_${task//,/_}-${shot}shot --batch_size ${batch_size} 2>&1 | tee /content/lm_eval_output/eval-${hf_model//\//_}_${task//,/_}-${shot}shot.log


apple/OpenELM-3B
mmlu,winogrande
5
2024-05-12 14:09:25.948337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 14:09:25.948379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 14:09:25.950387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 14:09:27.165210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-12:14:09:30,774 INFO     [__main__.py:254] Verbosity set to DEBUG
2024-05-12:14:09:37,154 INFO     [__main__.py:341] Selected Tasks: ['mmlu', 'winogrande']
2024-05-12:14:09:37,160 INFO     [evaluato

In [38]:
!pip install accelerate -q

In [116]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

#model_id = "apple/OpenELM-3B"
model_id = 'meta-llama/Llama-2-7b-hf'
model_id_Tokenizer = 'meta-llama/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id_Tokenizer)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", torch_dtype=torch.bfloat16, trust_remote_code=True, do_sample=True)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [120]:
prompt="What was the first album Beyoncé released as a solo artist?"
prompt="Once upon a time there was"

outputs = pipe(prompt, max_new_tokens=300, do_sample=True,  eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

In [122]:
print(outputs)

[{'generated_text': 'Once upon a time there was a man who lived in a little house on the edge of a village. The man was very old and had no children. He was very lonely and wished he had a friend. One day he went out into the forest to gather firewood. He found a beautiful little bird sitting in a tree. The bird was singing a beautiful song. The old man was so happy to hear the bird singing that he wanted to give the bird a gift. He picked up a stone and threw it at the bird. The bird flew away. The man felt so sad that he lay down under the tree and died. The bird was very sad too and flew back to the tree. When the bird saw the old man lying under the tree it sang a beautiful song. The song was so beautiful that the old man woke up. He was very happy to see the bird and he gave the bird a gift. The bird was so happy that it flew away and sang a beautiful song. The old man was so happy that he lay down under the tree and died. The bird was very sad and flew back to the tree. When the 

In [121]:
print('Question: %s'%prompt)
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: Once upon a time there was
Generated Answer:
a man who lived in a little house on the edge of a village. The man was very old and had no children. He was very lonely and wished he had a friend. One day he went out into the forest to gather firewood. He found a beautiful little bird sitting in a tree. The bird was singing a beautiful song. The old man was so happy to hear the bird singing that he wanted to give the bird a gift. He picked up a stone and threw it at the bird. The bird flew away. The man felt so sad that he lay down under the tree and died. The bird was very sad too and flew back to the tree. When the bird saw the old man lying under the tree it sang a beautiful song. The song was so beautiful that the old man woke up. He was very happy to see the bird and he gave the bird a gift. The bird was so happy that it flew away and sang a beautiful song. The old man was so happy that he lay down under the tree and died. The bird was very sad and flew back to the tree. Wh